In [66]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
# from pywebcopy import save_webpage

In [67]:

internal_urls = set()
external_urls = set()

In [68]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [69]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [70]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_urls(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)


In [71]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.american.edu/coronavirus/"

In [78]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.american.edu', path='/coronavirus/', params='', query='', fragment='')
https://www.american.edu
www.american.edu/coronavirus/
www.american.edu


In [56]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://www.american.edu/coronavirus/
[INT] Internal link: https://www.american.edu/
[INT] Internal link: https://www.american.edu/directory/
[INT] Internal link: https://www.american.edu/admissions/index.cfm
[INT] Internal link: https://www.american.edu/admissions/apply/index.cfm
[INT] Internal link: https://www.american.edu/admissions/freshman/index.cfm
[INT] Internal link: https://www.american.edu/programs/graduate/index.cfm
[INT] Internal link: https://www.american.edu/finance/studentaccounts/Tuition-and-Fees-Information.cfm
[INT] Internal link: https://www.american.edu/financialaid/index.cfm
[INT] Internal link: https://www.american.edu/academics/index.cfm
[INT] Internal link: https://www.american.edu/degrees.cfm
[INT] Internal link: https://www.american.edu/provost/registrar/studentservices/registration.cfm
[INT] Internal link: https://www.american.edu/provost/registrar/schedule/
[INT] Internal link: https://www.american.edu/summer/index.cfm
[INT] Internal li

[INT] Internal link: https://www.american.edu/library/forms/
[INT] Internal link: https://www.american.edu/library/forms/tel:2028853232
[INT] Internal link: https://www.american.edu/library/forms/tel:2028851000
[INT] Internal link: https://www.american.edu/library/forms/distance_ed.cfm
[INT] Internal link: https://www.american.edu/library/forms/media_easypass.cfm
[INT] Internal link: https://www.american.edu/library/forms/faculty_research_deposit.cfm
[INT] Internal link: https://www.american.edu/library/forms/priority_cataloging.cfm
[INT] Internal link: https://www.american.edu/library/forms/media_reserve.cfm
[INT] Internal link: https://www.american.edu/library/forms/serials_request.cfm
[INT] Internal link: https://www.american.edu/library/forms/syllabus_deposit.cfm
[INT] Internal link: https://www.american.edu/provost/undergrad/core/
[INT] Internal link: https://www.american.edu/provost/undergrad/core/Writing-Information-Literacy.cfm
[INT] Internal link: https://www.american.edu/prov

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: http://www.american.edu/ocl/healthcenter/upload/SHAC-Application.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.american.edu/ocl/student-health-faq.cfm
[INT] Internal link: https://www.american.edu/ocl/healthcenter/upload/Waiver-Requirements-2019-2020.pdf
[INT] Internal link: https://www.american.edu/ocl/healthcenter/upload/2019-Insurance-Comparison.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.american.edu/ocl/index.cfm
[INT] Internal link: https://www.american.edu/ocl/about-us.cfm
[INT] Internal link: https://www.american.edu/ocl/Meet-the-Staff.cfm
[INT] Internal link: https://www.american.edu/ocl/departments.cfm
[INT] Internal link: https://www.american.edu/ocl/memos.cfm
[INT] Internal link: https://www.american.edu/ocl/student-scholarships.cfm
[INT] Internal link: https://www.american.edu/ocl/places-to-study.cfm
[INT] Internal link: https://www.american.edu/universityawards/Students.cfm
[INT] Internal link: https://www.american.edu/ocl/social-media-tips.cfm
[INT] Internal link: https://www.american.edu/ocl/Civility.cfm
[INT] Internal link: https://www.american.edu/ocl/conduct.cfm
[INT] Internal link: https://www.american.edu/ocl/TitleIX/index.cfm
[INT] Internal link: https://www.american.edu/ocl/student-policies.cfm
[INT] Internal link: https://www.american.edu/ocl/bias-education.cfm
[INT] Internal link: https://www.american.edu/ocl/tel:20

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.american.edu/ocl/dos/
[INT] Internal link: https://www.american.edu/ocl/volunteer/index.cfm
[INT] Internal link: https://www.american.edu/ocl/cdi/index.cfm
[INT] Internal link: https://www.american.edu/ocl/counseling/
[INT] Internal link: https://www.american.edu/ocl/promote-health/index.cfm
[INT] Internal link: https://www.american.edu/ocl/housing/
[INT] Internal link: https://www.american.edu/ocl/isss/
[INT] Internal link: https://www.american.edu/ocl/kay/
[INT] Internal link: https://www.american.edu/ocl/orientation/
[INT] Internal link: https://www.american.edu/ocl/sccrs/
[INT] Internal link: https://www.american.edu/ocl/titleix/
[INT] Internal link: https://www.american.edu/ocl/conferences/index.cfm
[INT] Internal link: https://www.american.edu/ocl/dos/FAQ.cfm
[INT] Internal link: https://www.american.edu/ocl/dos/How-we-help-students.cfm
[INT] Internal link: https://www.american.edu/ocl/dos/index.cfm
[INT] Internal link: https://www.american.edu/oc

In [59]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,5):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        crawl(immediate_url)
        print("[+] NEW Total External links:", len(external_urls))
        print("[+] NEW Total Internal links:", len(internal_urls))
        print("[+] NEW Total:", len(external_urls) + len(internal_urls))

37 0
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW T

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
[+] NEW Total External links: 336
[+] NEW Total Internal links: 384
[+] NEW Total: 720
37 1


In [60]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [61]:
final_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    final_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)

print(final_immediate_urls)
total_count = len(final_immediate_urls)
print(total_count)

{'https://www.american.edu/coronavirus/resources.cfm', 'https://www.american.edu/coronavirus/faq-campus-life.cfm', 'https://www.american.edu/coronavirus/faq-general-questions.cfm', 'https://www.american.edu/coronavirus/fallgen6b', 'https://www.american.edu/coronavirus/faq-academics.cfm', 'https://www.american.edu/coronavirus/upload/AU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf', 'https://www.american.edu/coronavirus/students.cfm', 'https://www.american.edu/coronavirus/FAQ-Campus-Life.cfm', 'https://www.american.edu/coronavirus/FAQ-Academics.cfm', 'https://www.american.edu/coronavirus/Services-and-Hours.cfm', 'https://www.american.edu/coronavirus/FAQ-Travel-and-Study-Abroad.cfm', 'https://www.american.edu/coronavirus/upload/AUForward_HowWeWork_Guide_June16.pdf', 'https://www.american.edu/coronavirus/faq-travel-and-study-abroad.cfm', 'https://www.american.edu/coronavirus/au-forward.cfm', 'https://www.american.edu/coronavirus/FAQ-General-Questions.cfm', 'https://www.american.edu/coro

In [62]:
count = 0
parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
for i in final_immediate_urls:
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    url = i
    file_name = url.replace(base_url,'')
    file_name = file_name.replace('/','')
    if file_name == '':
            file_name = "Home"
    count = count + 1
    print(f"{count} of {total_count}")
    print(f'Visiting {base_url_text}/{file_name}')
    print(f'...Taking a screenshot')
    driver.get(url)
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
    driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
    print(f'Screenshot of {file_name} page taken! \n')
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
    driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

1 of 36
Visiting www.american.edu/coronavirus//resources.cfm
...Taking a screenshot
Screenshot of resources.cfm page taken! 

2 of 36
Visiting www.american.edu/coronavirus//faq-campus-life.cfm
...Taking a screenshot
Screenshot of faq-campus-life.cfm page taken! 

3 of 36
Visiting www.american.edu/coronavirus//faq-general-questions.cfm
...Taking a screenshot
Screenshot of faq-general-questions.cfm page taken! 

4 of 36
Visiting www.american.edu/coronavirus//fallgen6b
...Taking a screenshot
Screenshot of fallgen6b page taken! 

5 of 36
Visiting www.american.edu/coronavirus//faq-academics.cfm
...Taking a screenshot
Screenshot of faq-academics.cfm page taken! 

6 of 36
Visiting www.american.edu/coronavirus//uploadAU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf
...Taking a screenshot
Screenshot of uploadAU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf page taken! 

7 of 36
Visiting www.american.edu/coronavirus//students.cfm
...Taking a screenshot
Screenshot of students.cfm page taken! 

In [63]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    print(i)
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
#         print(code)
    except urllib.error.HTTPError as e:
        if e.code not in (400,403):
            webpage_cannot_save = webpage_cannot_save + 1
#             print("discarded")
            continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        
#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

https://www.american.edu/coronavirus/resources.cfm
https://www.american.edu/coronavirus/faq-campus-life.cfm
https://www.american.edu/coronavirus/faq-general-questions.cfm
https://www.american.edu/coronavirus/fallgen6b
https://www.american.edu/coronavirus/faq-academics.cfm
https://www.american.edu/coronavirus/upload/AU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf
https://www.american.edu/coronavirus/students.cfm
https://www.american.edu/coronavirus/FAQ-Campus-Life.cfm
https://www.american.edu/coronavirus/FAQ-Academics.cfm
https://www.american.edu/coronavirus/Services-and-Hours.cfm
https://www.american.edu/coronavirus/FAQ-Travel-and-Study-Abroad.cfm
https://www.american.edu/coronavirus/upload/AUForward_HowWeWork_Guide_June16.pdf
https://www.american.edu/coronavirus/faq-travel-and-study-abroad.cfm
https://www.american.edu/coronavirus/au-forward.cfm
https://www.american.edu/coronavirus/FAQ-General-Questions.cfm
https://www.american.edu/coronavirus/faq-finances-and-student-services.cfm
h

In [64]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    url = webpage
    file_name = url.replace(base_url,'')
    file_name = file_name.replace('/','')
    if file_name == '':
            file_name = "Home"
    urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
    webpage_save_counter_loop = webpage_save_counter_loop+1
    print(f'{webpage_save_counter_loop} of {webpage_save_count} - {webpage}')

print("Saving Completed")

1 of 33 - https://www.american.edu/coronavirus/resources.cfm
2 of 33 - https://www.american.edu/coronavirus/faq-campus-life.cfm
3 of 33 - https://www.american.edu/coronavirus/faq-general-questions.cfm
4 of 33 - https://www.american.edu/coronavirus/faq-academics.cfm
5 of 33 - https://www.american.edu/coronavirus/upload/AU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf
6 of 33 - https://www.american.edu/coronavirus/students.cfm
7 of 33 - https://www.american.edu/coronavirus/FAQ-Campus-Life.cfm
8 of 33 - https://www.american.edu/coronavirus/FAQ-Academics.cfm
9 of 33 - https://www.american.edu/coronavirus/Services-and-Hours.cfm
10 of 33 - https://www.american.edu/coronavirus/FAQ-Travel-and-Study-Abroad.cfm
11 of 33 - https://www.american.edu/coronavirus/upload/AUForward_HowWeWork_Guide_June16.pdf
12 of 33 - https://www.american.edu/coronavirus/faq-travel-and-study-abroad.cfm
13 of 33 - https://www.american.edu/coronavirus/au-forward.cfm
14 of 33 - https://www.american.edu/coronavirus/faq-

In [65]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    if webpage[-4:] == ".pdf":
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
            file_name = "Home"
        pdf_urls.add(webpage)
        print(webpage)
        r = requests.get(url, stream=True)
        with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
            for chunk in r.iter_content(2000):
                fd.write(chunk)
        pdf_count = pdf_count + 1

print(f"Pdfs saved: {pdf_count}")

https://www.american.edu/coronavirus/upload/AU-FORWARD-FALL-2020-PRESENTATION-FINAL_June16.pdf
https://www.american.edu/coronavirus/upload/AUForward_HowWeWork_Guide_June16.pdf
https://www.american.edu/coronavirus/upload/AUForward_Fall2020_HealthandSafetyPlan_June16.pdf
https://www.american.edu/coronavirus/upload/AU_TaskForceMembers_5-11-20.pdf
Pdfs saved: 4
